In [1]:
from executorch.extension.llm.runner import MultimodalRunner, GenerationConfig, make_image_input, make_text_input
from executorch.kernels import quantized
from transformers import AutoProcessor
from executorch.extension.llm.custom_ops import custom_ops

I tokenizers:regex.cpp:27] Registering override fallback regex
/Users/mengweiliu/miniconda3/envs/executorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0919 00:02:56.437000 17305 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
model_id = "mistralai/Voxtral-Mini-3B-2507"
processor = AutoProcessor.from_pretrained(model_id)
audio_url = "https://huggingface.co/datasets/eustlb/audio-samples/resolve/main/dude_where_is_my_car.wav"
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "audio", "url": audio_url},
            {
                "type": "text",
                "text": "What can you tell me about this audio?",
            },
        ],
    },
]
inputs = processor.apply_chat_template(conversation,
    tokenize=True,
    return_dict=True,
    return_tensors="pt")


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 20360.70it/s]


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.save_pretrained("/Volumes/larryliu/work/models/voxtral/")

In [3]:
l = inputs["input_ids"].tolist()

In [12]:
l[0][-2:]

[1063, 4]

In [7]:
tokenizer.decode(inputs["input_ids"].tolist()[0])

'<s>[INST][BEGIN_AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUDIO][AUD

In [17]:
tokenizer.encode("[BEGIN_AUDIO]")

[1, 1091, 91264, 10376, 84310, 1093, 2]

In [8]:
from executorch.extension.llm.runner import make_audio_input, make_token_input

In [5]:
inputs["input_features"].shape

torch.Size([3, 128, 3000])

In [ ]:
inputs_combined = [
    make_token_input([1, 3, 25]),
    make_audio_input(inputs["input_features"]), 
    make_text_input("\nWhat can you tell me about this audio?"),
]
runner = MultimodalRunner("/Volumes/larryliu/work/models/voxtral/model.pte", "/Volumes/larryliu/work/models/voxtral/tekken.json", None)
config = GenerationConfig()
config.max_new_tokens = 100
runner.generate(inputs_combined, config)

I tokenizers:tekken.cpp:88] Loading Tekken tokenizer from: /Volumes/larryliu/work/models/voxtral/tekken.json
I tokenizers:tekken.cpp:117] Tekken version: v7, vocab_size: 131072, special_tokens: 1000
I tokenizers:tekken.cpp:123] Loading special tokens from JSON
I tokenizers:tekken.cpp:287] Initialized 1000 special tokens (1000 defined, 0 placeholders)
I tokenizers:tekken.cpp:140] Loading 130072 vocabulary tokens
I tokenizers:tekken.cpp:227] Processing 130072 vocabulary entries (limit: 130072)
I tokenizers:tekken.cpp:260] Built vocabulary with 130072 tokens
E0000 00:00:1758265519.484524 137702822 re2.cc:237] Error parsing '([^\r\n\p{L}\p{N}]?[\p{Lu}\p{Lt}\p{Lm}\p{Lo}\p{M}]*[\p{Ll}\p{Lm}\p{Lo}\p{M}]+|[^\r\n\p{L}\p{N}]?[\p{...': invalid perl operator: (?!
E tokenizers:re2_regex.cpp:26] Failed to compile regex: ([^\r\n\p{L}\p{N}]?[\p{Lu}\p{Lt}\p{Lm}\p{Lo}\p{M}]*[\p{Ll}\p{Lm}\p{Lo}\p{M}]+|[^\r\n\p{L}\p{N}]?[\p{Lu}\p{Lt}\p{Lm}\p{Lo}\p{M}]+[\p{Ll}\p{Lm}\p{Lo}\p{M}]*|\p{N}| ?[^\s\p{L}\p{N}]+[\r

</s>Oh, sorry. Sorry.</s>

PyTorchObserver {"prompt_tokens":1139,"generated_tokens":7,"model_load_start_ms":1758265519513,"model_load_end_ms":1758265521463,"inference_start_ms":1758265521463,"inference_end_ms":1758265554533,"prompt_eval_end_ms":1758265554142,"first_token_ms":1758265554142,"aggregate_sampling_time_ms":1,"SCALING_FACTOR_UNITS_PER_SECOND":1000}


[text_token_generator.h:123] 
Reached to the end of generation
[stats.h:108] 	Prompt Tokens: 1139    Generated Tokens: 7
[stats.h:114] 	Model Load Time:		1.950000 (seconds)
[stats.h:124] 	Total inference time:		33.070000 (seconds)		 Rate: 	0.211672 (tokens/second)
[stats.h:132] 		Prompt evaluation:	32.679000 (seconds)		 Rate: 	34.854188 (tokens/second)
[stats.h:143] 		Generated 7 tokens:	0.391000 (seconds)		 Rate: 	17.902813 (tokens/second)
[stats.h:151] 	Time to first generated token:	32.679000 (seconds)
[stats.h:158] 	Sampling time over 1146 tokens:	0.001000 (seconds)


In [15]:
runner.reset()
inputs_combined = [
    make_token_input([1, 3, 25]),
    make_audio_input(inputs["input_features"]), 
    make_text_input("\nWhat can you tell me about this audio?"),
]
runner.generate(inputs_combined, config)

AttributeError: 'MultimodalRunner' object has no attribute 'reset'

In [15]:
from executorch.extension.pybindings.portable_lib import _load_for_executorch

In [16]:
module = _load_for_executorch("/Volumes/larryliu/work/models/voxtral/model.pte")

In [17]:
print(module.method_meta("audio_encoder"))

MethodMeta(name='audio_encoder', num_inputs=1, input_tensor_meta=['TensorInfo(sizes=[10, 128, 3000], dtype=Float, is_memory_planned=False, nbytes=15360000)'], num_outputs=1, output_tensor_meta=['TensorInfo(sizes=[1, 3750, 3072], dtype=Float, is_memory_planned=True, nbytes=46080000)'])


In [27]:
image = inputs["pixel_values"]

In [47]:
res = module.run_method("vision_encoder", [image])

In [48]:
print(res[0].shape)

torch.Size([1, 576, 4096])


In [33]:
from transformers import LlavaForConditionalGeneration
model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype="auto", device_map="auto")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.46s/it]


In [34]:
model.config.name_or_path

'llava-hf/llava-1.5-7b-hf'

In [35]:
"llava" in model.config.name_or_path

True

In [37]:
import torch
class VisionExportableModule(torch.nn.Module):
    def __init__(self, model: torch.nn.Module):
        super().__init__()
        self.model = model

    def prepare_export_inputs(self):
        # 1. Get export inputs
        model_id = self.model.config.name_or_path
        processor = AutoProcessor.from_pretrained(model_id)
        sample_conversation_with_image = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "url": "https://llava-vl.github.io/static/images/view.jpg"},
                ],
            },
        ]
        processed_inputs = processor.apply_chat_template(
            sample_conversation_with_image,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        )
        if "pixel_values" not in processed_inputs:
            raise ValueError(
                f"Unable to obtain sample audio encoder inputs for export for {model_id} - the processor did not return formatted inputs with the 'pixel_values' key: {processed_inputs}"
            )
        export_inputs = processed_inputs["pixel_values"]

        # 2. Get export dynamic shapes
        dynamic_shapes = None  # No batching for now.

        return export_inputs, dynamic_shapes

    def forward(
        self,
        input_features: torch.FloatTensor,
    ):
        image_embeds = self.model.get_image_features(input_features)
        # Hack, assuming the text decoder will take a 3D tensor (batch_size, seq_len, hidden_size)
        if "llava" in self.model.config.name_or_path:
            # Llava returns a list of 2D tensors (seq_len, hidden_size)
            return image_embeds[0].unsqueeze(0)
        else:
            return image_embeds

In [41]:
vision = VisionExportableModule(model.to("cpu"))

In [43]:
a = vision.forward(image.to("cpu"))

In [44]:
a.shape

torch.Size([1, 576, 4096])